In [1]:
import MDAnalysis as mda
from numpy.testing import assert_allclose

/home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from menger.data import files
from menger.tests.utils import retrieve_results

In [3]:
from menger.analysis.mengercurvature import MengerCurvature

In [15]:
u = mda.Universe(files.TUBULIN_CHAIN_ABC_PDB  , files.TUBULIN_CHAIN_ABC_DCD) 
calpha = u.select_atoms("name CA and chainID A")
md_name = "tubulin_chain_abc"
chainid = "A"

/home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [5]:
calpha

<AtomGroup with 450 atoms>

In [6]:
menger_analyser = MengerCurvature(u,"name CA and chainID A")
menger_analyser.run()


In [22]:
%%timeit
menger_analyser = MengerCurvature(u,"name CA")
menger_analyser.run()

114 ms ± 369 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
local_curvatures = menger_analyser.results.local_curvatures
local_flexibilities = menger_analyser.results.local_flexibilities

In [17]:
def make_filename(md_name : str, spacing : int , metric : str,chainid : str| None = None) -> str:

    filename : str = ""
    if chainid is None:
        filename=f"{md_name}_{metric}.npy"
    else:
       filename= f"{md_name}_spacing_{spacing}_chain_{chainid}_{metric}.npy"

    return filename

In [19]:
make_filename(md_name,spacing=1,metric="local_curvatures",chainid=chainid)

'tubulin_chain_abc_spacing_1_chain_A_local_curvatures.npy'

In [21]:
import os
import numpy as np
from menger.analysis.mengercurvature import MengerCurvature
from pathlib import Path

# Create test directory if it doesn't exist
output_dir = "../menger/data/test_data"
os.makedirs(output_dir, exist_ok=True)

results = []
spacings = range(1,6)

for i, spacing in enumerate(spacings):

    # run MengerCurvature analysis
    menger_analyser = MengerCurvature(u, "name CA", spacing=spacing)
    menger_analyser.run()# Get results for current spacing
    
    # Save local curvatures
    output_path = os.path.join(output_dir,make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="local_curvatures",
                                                        chainid=chainid))

    np.save(output_path, menger_analyser.results.local_curvatures)

    # Save local flexibilities
    output_path = os.path.join(output_dir, make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="local_flexibilities",
                                                        chainid=chainid))
    np.save(output_path, menger_analyser.results.local_flexibilities)
    
    # Save full menger array
    output_path = os.path.join(output_dir, make_filename(md_name=md_name,
                                                        spacing=spacing,
                                                        metric="menger_array",
                                                        chainid=chainid))
    np.save(output_path, menger_analyser.results.menger_array)


In [ ]:
output_dir

In [4]:
def test_menger_curvature(md_name  : str,universe: np.ndarray,select : str, spacing: int):
    menger_analyser = MengerCurvature(universe, select, spacing)
    menger_analyser.run()
    results = menger_analyser.results
    
    # test menger array 
    test_menger_array = results.menger_array
    expected_menger_array = retrieve_results(md_name, spacing, "menger_array")
    assert_allclose(test_menger_array,
                    expected_menger_array,
                    err_msg="Menger array is not as expected" + \
                    f" for md: {md_name} spacing {spacing}"
                    )

    # test local curvatures
    test_local_curvatures = results.local_curvatures
    expected_local_curvatures = retrieve_results(md_name, spacing, "local_curvatures")
    assert_allclose(test_local_curvatures,
                    expected_local_curvatures,
                    err_msg="Local curvatures are not as expected"+ \
                    f" for md: {md_name} spacing {spacing}"
                    )

    # test local flexibilities
    test_local_flexibilities = results.local_flexibilities
    expected_local_flexibilities = retrieve_results(md_name, spacing, "local_flexibilities")
    assert_allclose(test_local_flexibilities,
                    expected_local_flexibilities,
                    err_msg="Local flexibilities are not as expected"+ \
                    f" for md: {md_name} spacing {spacing}"
                        )

In [4]:
spacing = 2

In [ ]:
menger_analyser = MengerCurvature(u, "name CA", spacing)
menger_analyser.run()
results = menger_analyser.results

In [9]:
# md_name = "tubulin_chain_a"
expected_menger_array = retrieve_results(md_name, spacing, "menger_array")

In [ ]:
files.TEST_DATA_DIR

In [ ]:
expected_menger_array

In [11]:
test_menger_array = results.menger_array

In [ ]:
test_menger_array

In [8]:

# test menger array 
test_menger_array = results.menger_array

assert_allclose(test_menger_array,
                expected_menger_array,
                err_msg="Menger array is not as expected" + \
                f" for md: {md_name} spacing {spacing}"
                )


In [ ]:
! ls /home/etienne-reboul/anaconda3/envs/Menger_Curvature/lib/python3.10/site-packages/menger/data/test_data/

In [20]:
from menger.analysis.mengercurvature import menger_curvature

In [21]:
frame= np.array([[13.31, 34.22, 34.36],
                                [16.89, 33.47, 35.28],
                                [20.4 , 34.65, 34.76],
                                [23.99, 33.21, 34.96],
                                [27.52, 34.44, 34.73],
                                [31.27, 33.34, 35.16],
                                [34.95, 34.55, 34.84],
                                [38.57, 33.49, 35.07],
                                [42.11, 34.67, 34.64],
                                [45.72, 33.37, 34.84],
                                [49.49, 34.3 , 34.62],
                                [53.24, 33.33, 34.85],
                                [56.58, 35.18, 34.74]],
                                dtype=np.float32)

In [ ]:
np.round(menger_curvature(frame, 2), 2)